<a href="https://colab.research.google.com/github/DineshGujjeti/Infosys_Springboard_CodeGenie/blob/main/Milestone2/code/milestone2_code_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Install Dependencies**

In [ ]:
# @title Phase 1: Install Dependencies
!pip install transformers torch accelerate bitsandbytes pandas huggingface_hub radon ipywidgets matplotlib seaborn -q


# @title Phase 2: Imports and Hugging Face Login
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from huggingface_hub import notebook_login
import ipywidgets as widgets
from IPython.display import display, clear_output
from radon.complexity import cc_visit
from radon.metrics import mi_visit
import matplotlib.pyplot as plt
import pandas as pd
import re
from IPython.display import display, HTML
import io
import time
import numpy as np
from transformers import AutoModelForCausalLM, AutoTokenizer
import ast
import seaborn as sns
from radon.raw import analyze




print("Libraries imported successfully.")

# **Secure Hugging Face Login**

In [ ]:
notebook_login()

# **Configuration & Backend Engine**

In [ ]:
# --- Model Configuration ---
MODELS_TO_TEST = {
    "DeepSeek-Coder-1.3B": "deepseek-ai/deepseek-coder-1.3b-instruct",
    "Phi-2-2.7B": "microsoft/phi-2",
    "Gemma-2B-IT": "google/gemma-2b-it",
    "Stable-Code-3B": "stabilityai/stable-code-3b",      # Added
    "Replit-Code-3B": "replit/replit-code-v1-3b",        # Added
}
device = "cuda" if torch.cuda.is_available() else "cpu"

# --- Helper & Generation Functions ---
def clean_generated_code(text, model_path):
    model_path = model_path.lower()
    if "gemma" in model_path:
        text = re.sub(r"<start_of_turn>user\n.*<end_of_turn>\n<start_of_turn>model\n", "", text, flags=re.DOTALL).replace("<end_of_turn>", "")
    elif "phi-2" in model_path:
        text = re.sub(r"Instruct:.*\nOutput:", "", text, flags=re.DOTALL)
    elif "stable-code" in model_path or "replit" in model_path:
        text = re.sub(r"### Instruction:\n.*\n\n### Response:", "", text, flags=re.DOTALL)
    else:
        text = re.sub(r"### Instruction:\n.*\n\n### Response:", "", text, flags=re.DOTALL)
    match = re.search(r"``````", text, re.DOTALL)
    if match:
        text = match.group(1)
    return text.strip()

def is_syntactically_valid(code_string: str) -> bool:
    if not code_string:
        return False
    try:
        ast.parse(code_string)
        return True
    except SyntaxError:
        return False

def calculate_advanced_metrics(code_string):
    if not is_syntactically_valid(code_string):
        return {"complexity": None, "maintainability": None, "loc": None}
    try:
        complexity = sum([c.complexity for c in cc_visit(code_string)]) if cc_visit(code_string) else 0
        maintainability = mi_visit(code_string, multi=True)
        loc = analyze(code_string).loc
        return {"complexity": complexity, "maintainability": round(float(maintainability), 2), "loc": loc}
    except:
        return {"complexity": None, "maintainability": None, "loc": None}

def generate_code(model, tokenizer, prompt):
    model_path = model.name_or_path.lower()
    if "gemma" in model_path:
        formatted_prompt = f"<start_of_turn>user\n{prompt}<end_of_turn>\n<start_of_turn>model\n"
    elif "phi-2" in model_path:
        formatted_prompt = f"Instruct: {prompt}\nOutput:"
    elif "stable-code" in model_path or "replit" in model_path:
        formatted_prompt = f"### Instruction:\n{prompt}\n\n### Response:"
    else:
        formatted_prompt = f"### Instruction:\n{prompt}\n\n### Response:"
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    inputs = tokenizer(formatted_prompt, return_tensors="pt", return_attention_mask=True).to(device)
    start_time = time.time()
    output_ids = model.generate(
        inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_new_tokens=512,
        temperature=0.1,
        do_sample=True,
        pad_token_id=tokenizer.pad_token_id,
    )
    end_time = time.time()

    raw_output = tokenizer.batch_decode(output_ids)[0]
    cleaned_code = clean_generated_code(raw_output, model_path)

    return {"code": cleaned_code, "gen_time": end_time - start_time}

print("Backend engine with advanced metrics is ready.")


# **Pre-Loading All Models**

In [ ]:
# --- Section 4: Pre-Loading All Models ---
loaded_models = {}
print("Starting to pre-load all models...")
for model_name, model_path in MODELS_TO_TEST.items():
    print(f"\n--- Loading {model_name}... ---")
    try:
        tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
        model = AutoModelForCausalLM.from_pretrained(
            model_path,
            torch_dtype=torch.bfloat16,
            device_map="auto",
            trust_remote_code=True
        )
        loaded_models[model_name] = {"model": model, "tokenizer": tokenizer}
        print(f"✅ {model_name} loaded successfully.")
    except Exception as e:
        print(f"✗ FAILED to load {model_name}. Error: {e}")
print("\n" + "="*50 + "\nAll available models are pre-loaded.\n" + "="*50)


# **Sample Prompts**

In [ ]:
SAMPLE_PROMPTS = {
    "Python: Bubble Sort": "Write a Python function to sort a list of numbers using the bubble sort algorithm.",
    "Python: FastAPI Endpoint": "Create a simple FastAPI endpoint in Python that returns a JSON object with the message 'Hello, World!'.",
    "JavaScript: Fetch API": "Write a JavaScript function that uses the Fetch API to get data from 'https://api.example.com/data' and logs it to the console.",
    "HTML/CSS: Login Form": "Generate the HTML and CSS for a simple, centered login form with fields for username, password, and a submit button.",
    "SQL: Select Employees": "Write a SQL query to select the 'name' and 'salary' of all employees from the 'employees' table who work in the 'Engineering' department and have a salary greater than 80000.",
    "Python: Pandas DataFrame": "Write a Python script using pandas to create a DataFrame with columns 'Name' and 'Age', add three rows of data, and then print the DataFrame.",
    "Python: Class for a Car": "Create a Python class named 'Car' with a constructor that accepts 'make', 'model', and 'year'. Include a method that returns a formatted string like '2023 Toyota Camry'.",
    "Regex: Validate Email": "Provide a regular expression in Python to validate an email address.",
    "Docker: Python App": "Write a simple Dockerfile to containerize a Python application that has its dependencies in a requirements.txt file.",
    "Data Science: Plot with Matplotlib": "Generate Python code using Matplotlib to plot a simple sine wave from 0 to 2*pi."
}

# **UI #1 - Benchmark All Models**

In [ ]:
def model_inspect_ui():
    model_checkboxes = [widgets.Checkbox(value=True, description=name) for name in MODEL_IDS]
    prompt_dropdown = widgets.Dropdown(options=SAMPLE_PROMPTS, description='Prompt:')
    go_btn = widgets.Button(description='Generate Code')
    out_box = widgets.Output()

    def on_generate_clicked(b):
        with out_box:
            clear_output()
            selected = [box.description for box in model_checkboxes if box.value]
            results = evaluate_models(selected, [prompt_dropdown.value])
            for model in selected:
                res = results[model][0]
                print(f"Model: {model}")
                print(f"Generated Code:\n{res['code']}\n")
                print(f"Complexity: {res['complexity']}, Maintainability: {res['maintainability']}, LOC: {res['loc']}\n")

    go_btn.on_click(on_generate_clicked)

    ui = widgets.VBox(model_checkboxes + [prompt_dropdown, go_btn, out_box])
    display(ui)

# **UI #2 - Inspect Models with Checkboxes**

In [ ]:
def model_inspect_ui():
    model_checkboxes = [widgets.Checkbox(value=True, description=name) for name in MODELS_TO_TEST]
    prompt_dropdown = widgets.Dropdown(options=SAMPLE_PROMPTS, description='Prompt:')
    go_btn = widgets.Button(description='Generate Code')
    out_box = widgets.Output()
    def on_generate_clicked(b):
        with out_box:
            clear_output()
            selected = [box.description for box in model_checkboxes if box.value]
            results = evaluate_models(selected, [prompt_dropdown.value])
            for model in selected:
                res = results[model][0]
                print(f"Model: {model}\nGenerated Code:\n{res['code']}\n")
                print(f"Complexity: {res['complexity']}, Maintainability: {res['maintainability']}, LOC: {res['loc']}, Gen Time: {res['gen_time']}s\n")
    go_btn.on_click(on_generate_clicked)
    ui = widgets.VBox(model_checkboxes + [prompt_dropdown, go_btn, out_box])
    display(ui)

# **Visualization**

In [ ]:
def visualize_metrics(results):
    models = list(results.keys())
    complexity = []
    maintainability = []
    loc = []
    for model in models:
        c = [x['complexity'] for x in results[model] if x['complexity'] is not None]
        m = [x['maintainability'] for x in results[model] if x['maintainability'] is not None]
        l = [x['loc'] for x in results[model] if x['loc'] is not None]
        complexity.append(sum(c)/len(c) if c else 0)
        maintainability.append(sum(m)/len(m) if m else 0)
        loc.append(sum(l)/len(l) if l else 0)
    fig, axs = plt.subplots(1, 3, figsize=(18, 5))
    axs[0].bar(models, complexity); axs[0].set_title("Avg Complexity")
    axs[1].bar(models, maintainability); axs[1].set_title("Avg Maintainability")
    axs[2].bar(models, loc); axs[2].set_title("Avg LOC")
    plt.show()